In [1]:
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import nest_asyncio
import uvicorn
import requests

nest_asyncio.apply()

In [2]:
import os
import sys
import json
import torch
import argparse
import numpy as np
import warnings
warnings.filterwarnings(action='ignore', message='Too many lattice symmetries was found')

from pymatgen.ext.matproj import MPRester

# IOs
from pymatgen.core.structure import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.phase_diagram import PhaseDiagram, GrandPotentialPhaseDiagram
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
from pymatgen.core.periodic_table import Element, Species
from pymatgen.core.composition import Composition
from pymatgen.analysis.structure_analyzer import oxide_type
from pymatgen.entries.computed_entries import ComputedEntry, ComputedStructureEntry

# House code
sys.path.append('script')
from relaxer import TrajectoryObserver, M3gnetRelaxer, ChgnetRelaxer, MaceRelaxer


mpr = MPRester(api_key="dPcAQJZ6y1NZidGuvTerIPPFXHtsOb3E")

2024-06-23 20:42:25.376602: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-23 20:42:25.399554: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 20:42:25.399580: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 20:42:25.400410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-23 20:42:25.405135: I tensorflow/core/platform/cpu_feature_guar

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


/home/ubuntu/miniconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module named 'phonopy'
No module named 'phonopy'


In [3]:
def has_common_element(list1, list2):
    
    return not set(list1).isdisjoint(list2)


def read_json(fjson):
    
    with open(fjson) as f:
        return json.load(f)


def write_json(d, fjson):
    
    with open(fjson, 'w') as f:
        json.dump(d, f)

    return


def get_ehull(mpr, vasp_entry, mp_entries, compatibility):
    
    mp_entries = compatibility.process_entries(mp_entries)

    # phase diagram
    PD = PhaseDiagram(mp_entries + [vasp_entry])
    decomp_info = PD.get_decomp_and_e_above_hull(vasp_entry, allow_negative=True)
    e_above_hull = decomp_info[1]

    return e_above_hull


def get_test_structure(mpid='mp-510462'):
    
    mp_data = mpr.get_entry_by_material_id(material_id=[test_id])
    test_structure = mp_data[1].structure
    
    return test_structure


def get_chemsys_mpdata(structure):
    
    U_els = {'Co': 3.32, 'Cr': 3.7, 'Fe': 5.3, 'Mn': 3.9,
             'Mo': 4.38, 'Ni': 6.2, 'V': 4.2, 'W': 6.2}
    
    species = []
    potcar_spec = []

    for i in set(structure.species):
        species.append(i.name)

    chemical_space = '-'.join(species)
    mp_data = mpr.get_entries_in_chemsys(chemical_space)
    
    hubbards = {}
    if has_common_element(list(U_els.keys()), species):
        for specie in species:
            if specie in U_els.keys():
                hubbards[specie] = U_els[specie]
            else:
                hubbards[specie] = 0

    for d in mp_data:
        for j in d.parameters['potcar_spec']:
            if not j in potcar_spec:
                potcar_spec.append(j)
        if len(potcar_spec) == len(species):
            break

    return mp_data, potcar_spec, hubbards

def get_relaxation_result(structure, relaxer):
        
    mp_data, potcar_spec, hubbards = get_chemsys_mpdata(structure)
    
    atoms = structure.to_ase_atoms()
    # relaxer = MaceRelaxer()
    
    result = relaxer.relax(atoms, fmax=0.5)
    result['parameters'] = mp_data[0].parameters
    result['parameters']['potcar_spec'] = potcar_spec
    result['parameters']['hubbards'] = hubbards
    
    lattice = result['final_structure'].lattice.matrix
    species = [specie for specie in result['final_structure'].species]
    coords = [site.frac_coords for site in result['final_structure'].sites]
    structure = Structure(lattice=lattice, species=species, coords=coords)

    gga_entry = ComputedStructureEntry(structure=structure,
                                       energy=result['final_energy'],
                                       entry_id='llm_generation',
                                       composition=structure.composition.remove_charges(),
                                       parameters=result['parameters']
                                       )

    compat = MaterialsProject2020Compatibility()
    gga_entry = compat.process_entry(gga_entry)
    ehull = get_ehull(mpr, gga_entry, mp_data, compat)
    
    return result, ehull



In [4]:
# the request sent from the user is a string of the structure in POSCAR format, so we need to save it as file and then read it
def data_preprocessing(request):
    with open("POSCAR", "w") as f:
        f.write(request)
    
    structure = Structure.from_file("POSCAR")
    return structure

In [5]:
structure = 'Li4 Ga4 Se8 O24\n1.0\n  -5.3669739999999999    5.3669739999999999    5.0097290000000001\n   5.3669739999999999   -5.3669739999999999    5.0097290000000001\n   5.3669739999999999    5.3669739999999999   -5.0097290000000001\nLi Ga Se O\n4 4 8 24\ndirect\n   0.6892280000000000    0.8750000000000000    0.3142280000000000 Li\n   0.5607720000000000    0.3750000000000000    0.6857720000000000 Li\n   0.6250000000000000    0.3107720000000000    0.1857720000000000 Li\n   0.1250000000000000    0.4392280000000000    0.8142280000000000 Li\n   0.4126740000000000    0.8750000000000000    0.0376740000000000 Ga\n   0.8373260000000000    0.3750000000000000    0.9623260000000000 Ga\n   0.6250000000000000    0.5873260000000000    0.4623260000000000 Ga\n   0.1250000000000000    0.1626740000000000    0.5376740000000000 Ga\n   0.8557700000000000    0.1345510000000000    0.1717520000000000 Se\n   0.2872010000000000    0.6154490000000000    0.2212190000000000 Se\n   0.9627990000000000    0.6840180000000000    0.8282480000000000 Se\n   0.3942300000000000    0.0659820000000000    0.7787809999999999 Se\n   0.3159820000000000    0.1442300000000000    0.2787810000000000 Se\n   0.3845510000000000    0.6057700000000000    0.6717520000000000 Se\n   0.8654490000000000    0.0372010000000000    0.7212190000000001 Se\n   0.9340180000000000    0.7127990000000000    0.3282480000000000 Se\n   0.6802420000000000    0.0477010000000000    0.2209770000000000 O\n   0.4232760000000000    0.7022990000000000    0.1325410000000000 O\n   0.8267240000000000    0.4592650000000000    0.7790230000000000 O\n   0.5697580000000000    0.2907350000000000    0.8674590000000000 O\n   0.5407350000000000    0.3197580000000000    0.3674590000000000 O\n   0.2977010000000000    0.4302420000000000    0.7209770000000000 O\n   0.9522990000000000    0.1732760000000000    0.6325410000000000 O\n   0.7092650000000000    0.5767240000000000    0.2790230000000000 O\n   0.8402190000000000    0.3160960000000000    0.1584550000000000 O\n   0.0923590000000000    0.4339040000000000    0.0241230000000000 O\n   0.1576410000000000    0.6817640000000000    0.8415440000000000 O\n   0.4097810000000000    0.0682360000000000    0.9758770000000000 O\n   0.3182360000000000    0.1597810000000000    0.4758770000000000 O\n   0.5660960000000000    0.5902190000000000    0.6584550000000000 O\n   0.6839040000000000    0.8423590000000000    0.5241230000000000 O\n   0.9317640000000000    0.9076410000000000    0.3415440000000000 O\n   0.0426090000000000    0.2550480000000000    0.3775590000000000 O\n   0.3725100000000000    0.4949520000000000    0.2875600000000000 O\n   0.8774900000000000    0.6650500000000000    0.6224410000000000 O\n   0.2073910000000000    0.0849500000000000    0.7124400000000000 O\n   0.3349500000000000    0.9573910000000000    0.2124400000000000 O\n   0.5050480000000001    0.7926090000000000    0.8775590000000000 O\n   0.7449519999999999    0.1225100000000000    0.7875600000000000 O\n   0.9150500000000000    0.6274900000000000    0.1224410000000000 O\n'

data_preprocessing(structure)

Structure Summary
Lattice
    abc : 9.094295161627041 9.094295161627041 9.094295161627041
 angles : 107.66507236041433 107.66507236041433 113.14724687273069
 volume : 577.2091507498374
      A : -5.366974 5.366974 5.009729
      B : 5.366974 -5.366974 5.009729
      C : 5.366974 5.366974 -5.009729
    pbc : True True True
PeriodicSite: Li (2.683, 0.6894, 6.262) [0.6892, 0.875, 0.3142]
PeriodicSite: Li (2.683, 4.678, 1.252) [0.5608, 0.375, 0.6858]
PeriodicSite: Li (-0.6894, 2.683, 3.757) [0.625, 0.3108, 0.1858]
PeriodicSite: Li (6.056, 2.683, -1.252) [0.125, 0.4392, 0.8142]
PeriodicSite: Ga (2.683, -2.279, 6.262) [0.4127, 0.875, 0.03767]
PeriodicSite: Ga (2.683, 7.646, 1.252) [0.8373, 0.375, 0.9623]
PeriodicSite: Ga (2.279, 2.683, 3.757) [0.625, 0.5873, 0.4623]
PeriodicSite: Ga (3.088, 2.683, -1.252) [0.125, 0.1627, 0.5377]
PeriodicSite: Se (-2.949, 4.793, 4.101) [0.8558, 0.1346, 0.1718]
PeriodicSite: Se (2.949, -0.5744, 3.414) [0.2872, 0.6154, 0.2212]
PeriodicSite: Se (2.949, 5.941, 4.

In [6]:
Structure.from_file("POSCAR")

Structure Summary
Lattice
    abc : 9.094295161627041 9.094295161627041 9.094295161627041
 angles : 107.66507236041433 107.66507236041433 113.14724687273069
 volume : 577.2091507498374
      A : -5.366974 5.366974 5.009729
      B : 5.366974 -5.366974 5.009729
      C : 5.366974 5.366974 -5.009729
    pbc : True True True
PeriodicSite: Li (2.683, 0.6894, 6.262) [0.6892, 0.875, 0.3142]
PeriodicSite: Li (2.683, 4.678, 1.252) [0.5608, 0.375, 0.6858]
PeriodicSite: Li (-0.6894, 2.683, 3.757) [0.625, 0.3108, 0.1858]
PeriodicSite: Li (6.056, 2.683, -1.252) [0.125, 0.4392, 0.8142]
PeriodicSite: Ga (2.683, -2.279, 6.262) [0.4127, 0.875, 0.03767]
PeriodicSite: Ga (2.683, 7.646, 1.252) [0.8373, 0.375, 0.9623]
PeriodicSite: Ga (2.279, 2.683, 3.757) [0.625, 0.5873, 0.4623]
PeriodicSite: Ga (3.088, 2.683, -1.252) [0.125, 0.1627, 0.5377]
PeriodicSite: Se (-2.949, 4.793, 4.101) [0.8558, 0.1346, 0.1718]
PeriodicSite: Se (2.949, -0.5744, 3.414) [0.2872, 0.6154, 0.2212]
PeriodicSite: Se (2.949, 5.941, 4.

In [7]:
def inference(structure):
    
    result, ehull = get_relaxation_result(structure)
    
    total = {}
    trajectory = []
    aaa = AseAtomsAdaptor()
    
    for i in result['trajectory'].atoms_trajectory:
        relaxed_structure = aaa.get_structure(i)
        trajectory.append(relaxed_structure.as_dict())
        
    total['trajectory'] = trajectory
    total['ehull'] = ehull * 1000
 
    return total

In [ ]:
app = FastAPI()

# load the model
relaxer = MaceRelaxer()

class PoscarRequest(BaseModel):
    poscar: str

def data_preprocessing(request):
    with open("POSCAR", "w") as f:
        f.write(request)
    
    structure = Structure.from_file("POSCAR")
    return structure

def inference(structure):
    result, ehull = get_relaxation_result(structure, relaxer)
    
    total = {}
    trajectory = []
    aaa = AseAtomsAdaptor()
    
    for i in result['trajectory'].atoms_trajectory:
        relaxed_structure = aaa.get_structure(i)
        trajectory.append(relaxed_structure.as_dict())
        
    total['trajectory'] = trajectory
    total['ehull'] = ehull * 1000
 
    return total

@app.get("/ping")
async def ping():
    return JSONResponse(content={"message": "pong"})

@app.post("/predict")
async def predict(data: PoscarRequest):
    # try:
        # Preprocess the data
    structure = data_preprocessing(data.poscar)
    
    # Run inference
    result = inference(structure)
    
    # Clean up the temporary file
    if os.path.exists("POSCAR"):
        os.remove("POSCAR")
        
        return JSONResponse(content=result)
    # except Exception as e:
    #     # Clean up the temporary file in case of error
    #     if os.path.exists("POSCAR"):
    #         os.remove("POSCAR")
    #     error_detail = {
    #         "error": str(e),
    #         "traceback": traceback.format_exc()
    #     }
    #     return JSONResponse(status_code=500, content=error_detail)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8019)

/home/ubuntu/miniconda3/envs/llm/lib/python3.9/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/ubuntu/miniconda3/envs/llm/lib/python3.9/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/ubuntu/miniconda3/envs/llm/lib/python3.9/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/ubuntu/miniconda3/envs/llm/lib/python3.9/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


No dtype selected, switching to float64 to match model dtype.
MACE will run on cpu


INFO:     Started server process [15494]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8019 (Press CTRL+C to quit)
Retrieving ThermoDoc documents: 100%|██████████| 3168/3168 [00:14<00:00, 221.88it/s] 
